In [1]:
!pip install yfinance
!pip install forex_python
!pip install cpi
!pip install huggingface_hub
!pip install datasets
import yfinance as yf
from datetime import datetime, timedelta
from forex_python.converter import get_rate
import pandas as pd
import numpy as np
import cpi
from sklearn.preprocessing import MinMaxScaler

cpi.update()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 12.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

/usr/local/lib/python3.8/dist-packages/cpi/__init__.py:42: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  warnings.warn(StaleDataWarning())


In [2]:
# getting stock of NASDAQ 100 composite index

nasdaq = yf.Ticker("^IXIC")
nasdaq.info

{'exchange': 'NIM',
 'shortName': 'NASDAQ Composite',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EST',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '-18000000',
 'quoteType': 'INDEX',
 'symbol': '^IXIC',
 'messageBoardId': 'finmb_INDEXIXIC',
 'market': 'us_market',
 'previousClose': 10305.241,
 'regularMarketOpen': 10364,
 'twoHundredDayAverage': 11736.055,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': None,
 'regularMarketDayHigh': 10604.136,
 'navPrice': None,
 'averageDailyVolume10Day': 4348486000,
 'totalAssets': None,
 'regularMarketPreviousClose': 10305.241,
 'fiftyDayAverage': 10885.394,
 'trailingAnnualDividendRate': None,
 'open': 10364,
 'toCurrency': None,
 'averageVolume10days': 4348486000,
 'expireDate': None,
 'yield': None,
 'algorithm': None,
 'dividendRate': None,
 'exDividendDate': None,
 'beta': None,
 'circulatingSupply': None,
 'startDate': None,
 'regularMarketDayLow': 10265.039,
 'priceHint': 2,

In [3]:
# get historical market data
hist = nasdaq.history(start="2003-01-01", end='2022-10-31')

In [4]:
hist = hist.drop(columns=['Dividends', 'Stock Splits'])

In [5]:
hist.index[0]

Timestamp('2003-01-02 00:00:00-0500', tz='America/New_York')

In [6]:
inflation = []

for t in hist.index:
  inflation.append(get_rate("USD", "EUR", t))

In [8]:
len(inflation)

4992

In [9]:
# getting the cpi - Consumer Price Index

cpi_items_df = cpi.series.get(seasonally_adjusted=False).to_dataframe()
cpi_items_df = cpi_items_df[cpi_items_df['period_type']=='monthly']
cpi_items_df['date'] = pd.to_datetime(cpi_items_df['date'])
cpi_items_df = cpi_items_df.set_index('date')
cpi_df = cpi_items_df['value'].loc['2003':'2023']

In [10]:
cpi_col = []
for x in hist.index:
  ts = datetime(x.year, x.month, 1)
  cpi_col.append(cpi_df[ts])

In [13]:
hist['Inflation'] = inflation
hist['CPI'] = cpi_col

In [14]:
# checking for nll values
hist.isnull().sum()

Open         0
High         0
Low          0
Close        0
Volume       0
Inflation    0
CPI          0
dtype: int64

In [15]:
hist['Month'] = hist.index.month

In [16]:
hist

,Open,High,Low,Close,Volume,Inflation,CPI,Month
Date,,,,,,,,
2003-01-02 00:00:00-05:00,1346.930054,1384.910034,1336.979980,1384.849976,1287540000,0.957304,181.700,1
2003-01-03 00:00:00-05:00,1382.359985,1389.439941,1374.609985,1387.079956,1149590000,0.962279,181.700,1
2003-01-06 00:00:00-05:00,1390.189941,1428.650024,1390.089966,1421.319946,1567690000,0.953471,181.700,1
2003-01-07 00:00:00-05:00,1424.260010,1442.260010,1416.229980,1431.569946,1755760000,0.959233,181.700,1
2003-01-08 00:00:00-05:00,1423.290039,1424.119995,1399.060059,1401.069946,1457010000,0.963670,181.700,1
...,...,...,...,...,...,...,...,...
2022-10-24 00:00:00-04:00,10867.070312,10983.519531,10713.330078,10952.610352,4808710000,1.015125,298.012,10
2022-10-25 00:00:00-04:00,10996.969727,11210.379883,10996.969727,11199.120117,5141760000,1.014096,298.012,10
2022-10-26 00:00:00-04:00,10969.019531,11205.839844,10948.379883,10970.990234,5224440000,0.997705,298.012,10


In [17]:
hist['Quarter_end'] = np.where(hist['Month']%3==0,1,0)



In [18]:
hist.groupby('Quarter_end').mean()


,Open,High,Low,Close,Volume,Inflation,CPI,Month
Quarter_end,,,,,,,,
0,4883.940542,4916.312214,4848.754440,4885.383126,2.330358e+09,0.813674,229.402791,6.060066
1,4901.431055,4934.214838,4863.102691,4899.734537,2.383956e+09,0.812875,229.632353,7.384753


In [19]:
hist = hist.drop(columns=['Month'])

In [20]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [22]:
# uploading dataset to huggingface

from datasets import Dataset
dataset = Dataset.from_pandas(hist)
dataset.push_to_hub("marvmk/scalable_project")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
# testing loading dataset

from datasets import load_dataset
ds = load_dataset("marvmk/scalable_project")
dd = pd.DataFrame(ds['train'][:])
dd

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4992 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/marvmk___parquet/marvmk--scalable_project-81826193fe2a8b61/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

,Open,High,Low,Close,Volume,Inflation,CPI,Quarter_end,Date
0,1346.930054,1384.910034,1336.979980,1384.849976,1287540000,0.957304,181.700,0,2003-01-02 00:00:00-05:00
1,1382.359985,1389.439941,1374.609985,1387.079956,1149590000,0.962279,181.700,0,2003-01-03 00:00:00-05:00
2,1390.189941,1428.650024,1390.089966,1421.319946,1567690000,0.953471,181.700,0,2003-01-06 00:00:00-05:00
3,1424.260010,1442.260010,1416.229980,1431.569946,1755760000,0.959233,181.700,0,2003-01-07 00:00:00-05:00
4,1423.290039,1424.119995,1399.060059,1401.069946,1457010000,0.963670,181.700,0,2003-01-08 00:00:00-05:00
...,...,...,...,...,...,...,...,...,...
4987,10867.070312,10983.519531,10713.330078,10952.610352,4808710000,1.015125,298.012,0,2022-10-24 00:00:00-04:00
4988,10996.969727,11210.379883,10996.969727,11199.120117,5141760000,1.014096,298.012,0,2022-10-25 00:00:00-04:00
4989,10969.019531,11205.839844,10948.379883,10970.990234,5224440000,0.997705,298.012,0,2022-10-26 00:00:00-04:00
4990,10971.950195,10998.730469,10780.500000,10792.669922,4773520000,0.996314,298.012,0,2022-10-27 00:00:00-04:00
